In [20]:
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV

import torch
import torch.utils.data as Data

In [6]:
data_train = np.load('Dataset/data_train.npy')
X = data_train[:,:3]
Y = data_train[:,-1]
print(X.shape)
print(Y.shape)

(2400000, 3)
(2400000,)


In [7]:
standarder = StandardScaler()
X_train , X_test, Y_train, Y_test = train_test_split(X,Y, test_size= 0.2, random_state=0)
standarder.fit(X_train)
X_train = standarder.transform(X_train)
X_test = standarder.transform(X_test)
X_train = torch.from_numpy(X_train)
#X_test = torch.from_numpy(X_test)
Y_train = torch.from_numpy(Y_train)
#Y_test = torch.from_numpy(Y_test)

In [36]:
data_train = Data.TensorDataset(X_train, Y_train)
#data_test = Data.TensorDataset(X_test, Y_test)
BATCH_SIZE = 65536
loder_train = Data.DataLoader(data_train, batch_size= BATCH_SIZE, shuffle= True)
#loder_test = Data.DataLoader(data_test, batch_size= BATCH_SIZE, shuffle= False)

In [37]:
clf = RandomForestClassifier(random_state= 0)

In [38]:
clf.get_params().keys()

dict_keys(['bootstrap', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [39]:
for step,(img, label) in enumerate(loder_train):
    
    param_grid = {
        'criterion':['gini','entropy'],
        'n_estimators':[10,50,100,200],
        'max_depth':[5,10,20]
    }    
    grid_search = GridSearchCV(clf, param_grid, n_jobs = -1, verbose=2, cv= 5)
    grid_search.fit(img.numpy(), label.numpy())
    
    break

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  1.6min finished


In [40]:
grid_search.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [41]:
model = grid_search.best_estimator_

In [42]:
model.score(X_test,Y_test)

0.8589729166666666

In [44]:
from sklearn.externals import joblib
joblib.dump(model, 'model/randomforest.joblib')

['model/randomforest.joblib']